In [1]:
import pandas as pd
import numpy as np

Define the data dictionary

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float,
              'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float,
              'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 
              'sqft_lot':int, 'view':int}

import the data

In [3]:
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

Create new features:
* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) 
and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. 
Consequently this variable will mostly affect houses with many bedrooms.
* Taking square root of sqft_living will decrease the separation between big house and small house. 
The owner may not be exactly twice as happy for getting a house that is twice as big.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
sales['floors_square'] = sales['floors']*sales['floors']

Using the entire house dataset, learn regression weights using an L1 penalty of 5e2. 
Make sure to add "normalize=True" when creating the Lasso object. 

In [5]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [8]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coef_)-1

    # Get learned parameters as a list
    w = list(model.coef_)

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print np.poly1d(w)

Check which coefficients are non-0

In [9]:
print_coefficients(model_all)

Learned polynomial for degree 16:
           12             10         3
6.175e+04 x  + 2.475e+04 x  + 134.4 x


Search for a good L1 penalty using a validation set.

In [10]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

In [11]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors_square'] = validation['floors']*validation['floors']

Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7]:
* Learn a model on TRAINING data using the specified l1_penalty. 
Make sure to specify normalize=True in the constructor:
* Compute the RSS on VALIDATION for the current model (print or save the RSS)
* Report which L1 penalty produced the lower RSS on VALIDATION.

In [12]:
best_RSS = 0
best_l1_penalty = 0
count = 0
for l1_penalty in np.logspace(1, 7, num=13):
    model = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model.fit(training[all_features], training['price']) # learn weights
    #compute RSS
    predictions = model.predict(validation[all_features])
    residuals = validation['price'] - predictions
    redsiduals_squared = residuals **2
    RSS = redsiduals_squared.sum()
    if count == 0:
        best_l1_penalty=l1_penalty
        best_RSS=RSS 
        best_model= model
        count+=1
    else:
        if RSS < best_RSS :
            best_l1_penalty=l1_penalty
            best_RSS=RSS  
            best_model= model
    #print model results
    print("l1_penalty: %.6f, RSS $%.6f\n" % (l1_penalty , RSS ))
print ("Best l1_penalty: %.6f, RSS $%.6f\n" % (best_l1_penalty , best_RSS))

l1_penalty: 10.000000, RSS $398213327300134.125000

l1_penalty: 31.622777, RSS $399041900253348.187500

l1_penalty: 100.000000, RSS $429791604072558.437500

l1_penalty: 316.227766, RSS $463739831045119.687500

l1_penalty: 1000.000000, RSS $645898733633803.500000

l1_penalty: 3162.277660, RSS $1222506859427156.750000

l1_penalty: 10000.000000, RSS $1222506859427156.750000

l1_penalty: 31622.776602, RSS $1222506859427156.750000

l1_penalty: 100000.000000, RSS $1222506859427156.750000

l1_penalty: 316227.766017, RSS $1222506859427156.750000

l1_penalty: 1000000.000000, RSS $1222506859427156.750000

l1_penalty: 3162277.660168, RSS $1222506859427156.750000

l1_penalty: 10000000.000000, RSS $1222506859427156.750000

Best l1_penalty: 10.000000, RSS $398213327300134.125000



Limit the number of nonzero weights
What if we absolutely wanted to limit ourselves to, say, 7 features? 
This may be important if we want to derive "a rule of thumb" --- an interpretable model that 
has only a few features in them.
In this section, we are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of l1_penalty values to find a narrow region of l1_penalty 
values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for l1_penalty that achieves
the desired sparsity. Here, we will again use a validation set to choose the best value for l1_penalty.

In [13]:
max_nonzeros = 7

For l1_penalty in np.logspace(1, 4, num=20):

* it a regression model with a given l1_penalty on TRAIN data. 
Add "alpha=l1_penalty" and "normalize=True" to the parameter list.
* Extract the weights of the model and count the number of nonzeros. 
Take account of the intercept adding 1 whenever the intercept is nonzero. Save the number of nonzeros to a list.

In [21]:
l1_penalty_min = 0
l1_penalty_max = 0
count_max=0
count_min=0
for l1_penalty in np.logspace(1, 4, num=20):
    #learn the weights on training
    model_all = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_all.fit(training[all_features], training['price']) # learn weights
    non_zeros = np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_)
    print("l1_penalty: %.6f, Non Zeros %.6f\n" % (l1_penalty , non_zeros ))
    if non_zeros > max_nonzeros:
        if count_max == 0 :
            l1_penalty_max = l1_penalty
            max_model = model_all
            count_max+=1
        elif l1_penalty > l1_penalty_max:
            l1_penalty_max = l1_penalty
            max_model = model_all
        print("Max l1_penalty: %.6f, Non Zeros %.6f\n"% ( l1_penalty_max, non_zeros))
    if non_zeros < max_nonzeros:
        if count_min == 0 :
            l1_penalty_min = l1_penalty
            min_model =model_all
            count_min+=1
        elif l1_penalty < l1_penalty_min:
            l1_penalty_min = l1_penalty
            min_model =model_all
        print("Min l1_penalty: %.6f, Non Zeros %.6f\n"% ( l1_penalty_min, non_zeros))
print("Min l1_penalty: %.6f, Non Zeros %.6f\nMax l1_penalty %.6f, Non Zeros %.6f\n" % 
      ( l1_penalty_min, np.count_nonzero(min_model.coef_) + np.count_nonzero(min_model.intercept_),
       l1_penalty_max, np.count_nonzero(max_model.coef_) + np.count_nonzero(max_model.intercept_)))

l1_penalty: 10.000000, Non Zeros 15.000000

Max l1_penalty: 10.000000, Non Zeros 15.000000

l1_penalty: 14.384499, Non Zeros 15.000000

Max l1_penalty: 14.384499, Non Zeros 15.000000

l1_penalty: 20.691381, Non Zeros 15.000000

Max l1_penalty: 20.691381, Non Zeros 15.000000

l1_penalty: 29.763514, Non Zeros 15.000000

Max l1_penalty: 29.763514, Non Zeros 15.000000

l1_penalty: 42.813324, Non Zeros 13.000000

Max l1_penalty: 42.813324, Non Zeros 13.000000

l1_penalty: 61.584821, Non Zeros 12.000000

Max l1_penalty: 61.584821, Non Zeros 12.000000

l1_penalty: 88.586679, Non Zeros 11.000000

Max l1_penalty: 88.586679, Non Zeros 11.000000

l1_penalty: 127.427499, Non Zeros 10.000000

Max l1_penalty: 127.427499, Non Zeros 10.000000

l1_penalty: 183.298071, Non Zeros 7.000000

l1_penalty: 263.665090, Non Zeros 6.000000

Min l1_penalty: 263.665090, Non Zeros 6.000000

l1_penalty: 379.269019, Non Zeros 6.000000

Min l1_penalty: 263.665090, Non Zeros 6.000000

l1_penalty: 545.559478, Non Zeros 

We now explore the region of l1_penalty we found: between ‘l1_penalty_min’ and ‘l1_penalty_max’. 
We look for the L1 penalty in this range that produces exactly the right number of nonzeros 
and also minimizes RSS on the VALIDATION set.

For l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):

1. Fit a regression model with a given l1_penalty on TRAIN data. 
As before, use "alpha=l1_penalty" and "normalize=True".
2. Measure the RSS of the learned model on the VALIDATION set
Find the model that the lowest RSS on the VALIDATION set and has sparsity equal to ‘max_nonzeros’. 


In [23]:
best_l1_penalty=0
best_RSS=0
count=0
predictions = 0
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    #learn the weights on training
    model_all = linear_model.Lasso(alpha=l1_penalty, normalize=True)
    model_all.fit(training[all_features], training['price']) # learn weights
    non_zeros = np.count_nonzero(model_all.coef_) + np.count_nonzero(model_all.intercept_)
    print("l1_penalty: %.6f, Non Zeros %.6f\n" % (l1_penalty , non_zeros ))
    if non_zeros == max_nonzeros:
        #compute the RSS on validation set    
        predictions = model_all.predict(validation[all_features])
        residuals = validation['price']-predictions
        residuals_squared = residuals**2
        RSS = residuals_squared.sum()
        if count == 0:
            best_l1_penalty=l1_penalty
            best_RSS=RSS 
            best_model= model_all
            count+=1
        else:
            if RSS < best_RSS :
                best_l1_penalty=l1_penalty
                best_RSS=RSS  
                best_model= model_all
        #print model results
        print("l1_penalty: %.6f, RSS $%.6f\n" % (l1_penalty , RSS ))
print ("Best l1_penalty: %.6f, RSS $%.6f\n" % (best_l1_penalty , best_RSS))

l1_penalty: 263.665090, Non Zeros 6.000000

l1_penalty: 256.494690, Non Zeros 6.000000

l1_penalty: 249.324291, Non Zeros 6.000000

l1_penalty: 242.153891, Non Zeros 6.000000

l1_penalty: 234.983492, Non Zeros 6.000000

l1_penalty: 227.813092, Non Zeros 6.000000

l1_penalty: 220.642693, Non Zeros 6.000000

l1_penalty: 213.472293, Non Zeros 6.000000

l1_penalty: 206.301894, Non Zeros 6.000000

l1_penalty: 199.131494, Non Zeros 7.000000

l1_penalty: 199.131494, RSS $445230739842614.062500

l1_penalty: 191.961094, Non Zeros 7.000000

l1_penalty: 191.961094, RSS $444239780526140.750000

l1_penalty: 184.790695, Non Zeros 7.000000

l1_penalty: 184.790695, RSS $443296716874312.812500

l1_penalty: 177.620295, Non Zeros 7.000000

l1_penalty: 177.620295, RSS $442406413188665.562500

l1_penalty: 170.449896, Non Zeros 7.000000

l1_penalty: 170.449896, RSS $441566698090139.000000

l1_penalty: 163.279496, Non Zeros 7.000000

l1_penalty: 163.279496, RSS $440777489641605.250000

l1_penalty: 156.109097